In [2]:
import numpy as np

def convolution3D(input_tensor, weight, bias=None, stride=1, padding=0):
    """
    3D свертка для входного тензора.

    Параметры:
    - input_tensor: Входной тензор в форме 'NHWDC' (где N - количество образцов, H - высота, W - ширина,
                   D - глубина и C - количество каналов).
    - weight: Ядро свертки в форме 'DHWC' (где D - глубина ядра, H - высота ядра, W - ширина ядра и C - количество
              входных каналов).
    - bias: Смещение (по умолчанию None).
    - stride: Шаг свертки (по умолчанию 1).
    - padding: Значение 'same' паддинга (по умолчанию 0).

    Возвращает:
    - Выходной тензор после свертки.
    """
    if len(input_tensor.shape) != 5 or len(weight.shape) != 5:
        raise ValueError("Неверные размеры входного тензора или ядра")

    batch_size, in_channels, input_depth, input_height, input_width = input_tensor.shape
    out_channels, _, kernel_depth, kernel_height, kernel_width = weight.shape

    # Проверяем, что число входных каналов совпадает
    if in_channels != weight.shape[1]:
        raise ValueError("Число входных каналов в тензоре и ядре должно совпадать")

    # Определяем размер выходного тензора после свертки с учетом 'same' padding
    output_depth = (input_depth + 2 * padding - kernel_depth) // stride + 1
    output_height = (input_height + 2 * padding - kernel_height) // stride + 1
    output_width = (input_width + 2 * padding - kernel_width) // stride + 1

    # Вычисляем паддинг для каждого измерения
    pad_depth = max(0, ((output_depth - 1) * stride + kernel_depth - input_depth))
    pad_height = max(0, ((output_height - 1) * stride + kernel_height - input_height))
    pad_width = max(0, ((output_width - 1) * stride + kernel_width - input_width))

    # Добавляем паддинг к входному тензору с использованием np.pad
    padded_input = np.pad(input_tensor, ((0, 0), (0, 0), (pad_depth // 2, (pad_depth + 1) // 2),
                                         (pad_height // 2, (pad_height + 1) // 2), (pad_width // 2, (pad_width + 1) // 2)),
                          mode='constant')

    # Инициализируем выходной тензор
    output_tensor = np.zeros((batch_size, out_channels, output_depth, output_height, output_width))

    # Проходим по батчам, каналам, глубине, высоте и ширине выходного тензора
    for b in range(batch_size):
        for o_c in range(out_channels):
            for d_out in range(output_depth):
                for h_out in range(output_height):
                    for w_out in range(output_width):
                        d_start = d_out * stride
                        d_end = d_start + kernel_depth
                        h_start = h_out * stride
                        h_end = h_start + kernel_height
                        w_start = w_out * stride
                        w_end = w_start + kernel_width

                        # Вычисляем свертку для каждого пикселя выходного тензора
                        output_tensor[b, o_c, d_out, h_out, w_out] = np.sum(
                            padded_input[b, :, d_start:d_end, h_start:h_end, w_start:w_end] * weight[o_c, :, :, :, :]
                        )

    # Добавляем bias, если он предоставлен
    if bias is not None:
        output_tensor += bias.reshape((1, -1, 1, 1, 1))

    return output_tensor

In [3]:
# Создаем случайные входные данные
batch_size = 1
input_depth = 5
input_height = 5
input_width = 5
in_channels = 3
input_tensor = np.random.rand(batch_size, in_channels, input_depth, input_height, input_width)

# Создаем случайные веса свертки
out_channels = 2
kernel_depth = 3
kernel_height = 3
kernel_width = 3
weight = np.random.rand(out_channels, in_channels, kernel_depth, kernel_height, kernel_width)

# Создаем случайное смещение (bias)
bias = np.random.rand(out_channels)

# Задаем параметры свертки
stride = 1
padding = 1

# Применяем вашу функцию
result = convolution3D(input_tensor, weight, bias, stride, padding)

# Выводим результат
print("Входной тензор:")
print(input_tensor)
print("\nВеса свертки:")
print(weight)
print("\nСмещение (bias):")
print(bias)
print("\nРезультат свертки:")
print(result)

Входной тензор:
[[[[[0.91505527 0.99390275 0.32802641 0.95020832 0.48186677]
    [0.8269968  0.20424169 0.8014964  0.79899184 0.6787237 ]
    [0.250975   0.51079695 0.5015805  0.46884737 0.07493255]
    [0.31627399 0.81912183 0.14418336 0.01405122 0.98003898]
    [0.04100132 0.27872822 0.94359717 0.25471781 0.81098619]]

   [[0.64174112 0.52830965 0.63910545 0.010802   0.54628122]
    [0.74160722 0.11356898 0.80983824 0.36219828 0.09248587]
    [0.9118833  0.53074089 0.92648756 0.50912507 0.98903802]
    [0.86182837 0.10123677 0.11793532 0.46061167 0.47494269]
    [0.99871058 0.06344798 0.88955992 0.01987549 0.3423178 ]]

   [[0.3155519  0.97689804 0.68441504 0.0591048  0.83085826]
    [0.06987309 0.59394343 0.30834504 0.11942113 0.71465245]
    [0.49521199 0.16117453 0.78723861 0.8510727  0.84997675]
    [0.47276259 0.01683196 0.606419   0.88586792 0.37997889]
    [0.9175627  0.25615192 0.47575458 0.90350988 0.33460143]]

   [[0.17498686 0.97966573 0.26112899 0.60208063 0.38578657]
  

In [20]:
import unittest
import numpy as np
from numpy.testing import assert_allclose

class TestConvolution3D(unittest.TestCase):

    def test_result_shape(self):
        # Тест проверяет, что форма выходного тензора правильная
        input_tensor = np.random.rand(2, 3, 4, 4, 4)
        weight = np.random.rand(4, 3, 3, 3, 3)
        bias = np.random.rand(4)
        output_tensor = convolution3D(input_tensor, weight, bias)
        self.assertEqual(output_tensor.shape, (2, 4, 2, 2, 2))  # Пример формы для этого теста

    def test_no_bias(self):
        # Тест проверяет, что результат без bias совпадает с результатом без bias из PyTorch
        input_tensor = np.random.rand(2, 3, 4, 4, 4)
        weight = np.random.rand(4, 3, 3, 3, 3)
        output_tensor_custom = convolution3D(input_tensor, weight)
        
        import torch
        input_tensor_torch = torch.tensor(input_tensor, dtype=torch.float32)
        weight_torch = torch.tensor(weight, dtype=torch.float32)
        output_tensor_torch = torch.nn.functional.conv3d(input_tensor_torch, weight_torch).numpy()

        assert_allclose(output_tensor_custom, output_tensor_torch, rtol=1e-5, atol=1e-8)

    def test_padding(self):
        # Тест проверяет, что результат с 'same' padding совпадает с результатом из PyTorch
        input_tensor = np.random.rand(2, 3, 4, 4, 4)
        weight = np.random.rand(4, 3, 3, 3, 3)
        bias = np.random.rand(4)
        output_tensor_custom = convolution3D(input_tensor, weight, bias, padding=1)
        
        import torch
        input_tensor_torch = torch.tensor(input_tensor, dtype=torch.float32)
        weight_torch = torch.tensor(weight, dtype=torch.float32)
        bias_torch = torch.tensor(bias, dtype=torch.float32)
        output_tensor_torch = torch.nn.functional.conv3d(input_tensor_torch, weight_torch, bias=bias_torch, padding=1).numpy()

        assert_allclose(output_tensor_custom, output_tensor_torch, rtol=1e-5, atol=1e-8)



In [21]:
unittest.main(argv=[''], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.008s

OK
